<img src="./images/DLI_Header.png" style="width: 400px;">

# 2.0 멀티-GPU 훈련 전략
이번 실습 노트북에서는 트랜스포머 기반 언어 모델을 훈련하기 위해  [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) NVIDIA 라이브러리를 사용한 분산 학습 전략 및 실험에 대한 기본 지식을 소개합니다.


### 학습 목표

이번 실습 노트북의 목표는 다음과 같습니다. |
* 분산 훈련 전략의 메커니즘 이해하기
* 데이터와 텐서 병렬 분산을 적용하여 1 노드에서 Megatron-LM 스크립트를 사용하여 간단한 분산 학습을 실행하기
* Megatron-LM 로그의 기본 출력 값을 이해하기


**[2.1 분산 학습 전략 소개](#1.1-The-hardware-overview)<br>**
**[2.2 Megatron-LM GPT 사전훈련을 활용한 단일 GPU 학습 실행](#1.1-The-hardware-overview)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.1 GPT 사전 훈련 스크립트 체크하기](#1.2.1-Exercise:-Explore-the-Test-Set)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.1 GPU 사전 훈련 스크립트 실행하기](#1.2.1-Exercise:-Explore-the-Test-Set)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.3 Megatron-LM 실행 로그 이해하기](#1.2.1-Exercise:-Explore-the-Test-Set)<br>
**[2.3 Megatron-LM GPT 사전훈련을 활용한 멀티 GPU 학습 실행](#1.2-The-SLURM-Cluster-overview)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.1 연습: 2 GPU에서 Megatron-LM GPT 사전 훈련 실행](#1.2.1-Exercise:-Explore-the-Test-Set)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.2 멀티-GPU Megatron-LM 실행 로그 이해하기](#1.2.1-Exercise:-Explore-the-Test-Set)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.3 모델 분산 시 고려할 점](#1.2.1-Exercise:-Explore-the-Test-Set)<br>

### 이전 실행/보류 중인 작업을 취소합니다.

다음으로 이동하기 전에 SLURM 대기열에서 아직 실행 중이거나 대기 중인 작업이 없는지 확인하십시오. 다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다:

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


여전히 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하여 `scancel` 명령어를 사용하여 모든 사용자의 작업을 취소합니다.

In [2]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 2.1 분산 학습 전략 소개
분산 학습 모드에서는 훈련 프로세스를 분할하는 것이 목표입니다. ---
# 2.1 분산 학습 전략 소개
분산 학습 모드에서 목표는 여러 기기에 걸쳐 훈련 프로세스를 분할하는 것입니다. 가장 일반적으로 사용되는 분산 전략은 **데이터* 및 **모델** 병렬화입니다.


## 데이터 분산 (Data Distribution)

신경망은 일반적으로 [Stochastic Gradient Descent - 확률적 경사 하강법](https://developer.nvidia.com/blog/a-data-scientists-guide-to-gradient-descent-and-backpropagation-algorithms/) 을 사용하여 순차적으로 처리되는 배치로 분할하는 방식으로 구성됩니다. 순방향 단계에서는 피쳐 맵이 계산되고 역방향 패스에서는 그레디언트가 계산되고 평균화되어 파라미터 업데이트가 결정됩니다. 마지막으로 모델의 매개 변수가 업데이트되면 다음 데이터 배치가 처리됩니다.

<img src="images/data_parallel.png" width="600"/>

데이터 병렬 모드에서 데이터는 여러 기기에 걸쳐 분할되며, 각 기기는 각 시스템에 의해 호스팅되는 동일한 신경망의 복사본에 의해 처리됩니다. 매개 변수 업데이트는 모든 기기에서 평균화되고 모델 업데이트는 모든 복사본에 반영됩니다.

더 많은 프로세서 (또는 더 높은 데이터 병렬화)를 사용하면 에폭(즉, 전체 데이터 세트) 시간이 줄어들기 때문에, 이는 학습 속도를 높이는 효과가 있습니다. 또한 업데이트된 그레디언트는 (글로벌 배치 크기 증가로 인해) 더 많은 수의 표본을 효과적으로 확인되기 때문에 수렴 시간에 긍정적인 영향을 미칩니다. 배치당 걸리는 시간은 그레디언트 교환 통신 비용이 추가되어 여전히 동일합니다. 

그레디언트 교환을 구현하기 위해서는 다양한 전략이 있습니다. 
- **Centralized** 방식으로 서버 시스템이 데이터 청크를 배포하고 그레디언트를 누적하며 모델 매개 변수를 업데이트합니다. 
- **Decentralized** 방식으로, 각 워커가 다른 워커로부터 그레디언트를 보내고 수집하여 모델의 매개 변수를 로컬로 집계하고 업데이트합니다. 또한 워커들은 서로 다른 속도로 연산을 수행할 수 있습니다. 따라서 모델 매개 변수는 워커의 동기화 포인트를 기반으로 동기식으로 업데이트할 수 있습니다. 또한 워커가 오래된 매개 변수를 사용하여 작업할 수 있도록 여유 있는 전략을 사용할 수 있습니다. 이 전략은 훈련 중에 불일치를 초래할 수도 있습니다.

여러 라이브러리는 데이터 병렬화 구현을 제공하며, 대표적인 예로 [Horovod](https://github.com/horovod/horovod)가 있습니다. Horovod는 TensorFlow, Keras, PyTorch, Apache MXNet 과 같은 여러 딥 러닝 프레임워크와 호환됩니다. NVIDIA APEX](https://nvidia.github.io/apex/)는 분산 학습 및 혼합 정밀도를 간소화하는 유틸리티를 제공하는 Pytorch 확장 라이브러리입니다.




## 모델 분산 전략

모델 병렬화는 여러 기기에 걸쳐 모델의 매개 변수를 분할하는 프로세스입니다. 이를 통해 피쳐 맵 교환으로 인한 추가 통신 비용으로 1개의 GPU에 맞지 않는 더 큰 모델을 학습할 수 있습니다. 

두 가지 유형의 모델 분산를 구별할 수 있습니다.

### 파이프라인 병렬화


<img src="images/pipeline_parallel1.png" width="600"/>

파이프라인 병렬화는 모델을 조각으로 순차적으로 잘라내고 각 부분을 특정 워커에게 할당하는 프로세스입니다. 예를 들어 device_1의 레이어 1,2와 device_2의 레이어 3,4 등이 있습니다. 

마이크로배치 파이프라인 병렬화 [GPipe](https://arxiv.org/pdf/1811.06965.pdf) 와 같은 다양한 파이프라인 전략이 있으며, GPipe는 기기가 다른 피어들이 출력을 통신하기를 기다리는 시간을 최소화하기 위해 사용되는 모델 파이프라이닝 최적화된 구현 방식입니다. 데이터 청크를 마이크로 배치로 분할하여 서로 다른 시스템이 서로 다른 마이크로 배치를 동시에 처리할 수 있도록 합니다.

![title](images/pipeline_parallel.png)

[Interleaved pipeline parallelism](https://github.com/NVIDIA/Megatron-LM/)은 장치당 하나의 순차적 레이어 집합 대신 각각 계산량이 적은 여러 개의 파이프라인 단계를 할당합니다. 
예를 들어 device_1의 레이어 1, 2, 9,10, device_2의 레이어 3, 4, 11,12 등이 있습니다. 


### 텐서 병렬화

<img src="images/tensor_parallel1.png" width="500"/>

텐서 병렬화는  워커들 간에 행렬 연산을 나누는 과정입니다. [Megatron-LM](https://github.com/NVIDIA/Megatron-LM/) 은 NVIDIA의 오픈 소스 라이브러리로, 혼합 정밀도를 사용하여 트랜스포머 기반 네트워크의 효율적인 학습 및 GPT, BERT, T5와 같은 트랜스포머 기반 모델의 멀티 노드 사전 훈련을 제공합니다. Megatron의 트랜스포머 구현에서 셀프 어텐션 및 MLP 연산은 트랜스포머 셀당 총 4개의 All-Reduce 연산으로 병렬 블록으로 나뉩니다 (포워드 패스에 2개, 백워드 패스에 2개).

<img src="images/tensor_parallel.png" width="250"/>

Megatron-LM 은 PyTorch 위에 구축되며 혼합 정밀도를 사용하여 GPT 및 BERT 변압기 아키텍처의 사전 훈련을 위해 데이터, 파이프라인 및 텐서 병렬화를 통합합니다.

---
# 2.2 Megatron-LM GPT 사전 훈련의 단일 GPU 훈련 실행 

먼저 간단한 Megatron-LM GPT 실행 스크립트를 살펴보겠습니다.

분산 학습 모드의 경우 스크립트는 [PyTorch distributed launcher](https://pytorch.org/docs/stable/distributed.html)를 사용합니다. PyTorch 분산 모듈은 Python 플래그 `-m torch.distributed.launch` 함께 사용됩니다. 

리소스는`--nnodes` 및`--nproc_per_node`  인자로 구성되어 노드 수 및 노드 당 사용할 GPU 수를 각각 지정합니다.

Megatron-LM 라이브러리를 사용하면 두 가지 유형의 분산 데이터 병렬 구현이 가능합니다. 
- `local` 은 백 프롭(back propagation) 단계 끝에에서 그래디언트 올-리듀스(gradient all-reduce)를 수행합니다. 
- `torch` 는 그래디언트 감소 연산과 백 프롭 연산(더 큰 모델 크기에서는 더 효율적임)을 겹치는 분산 데이터 병렬 래퍼입니다.

이 섹션에서는 해당 인자를 사용하여 Megatron-LM pretrain_gpt.py 스크립트를 실행하여 1 GPU에서 간단한 Megatron-LM GPT 사전 훈련 실행문을 돌려봅니다.


<img src="images/Megatron_run.PNG" width="600"/>
우리는  `--DDP-impl` 인자를 사용하여 분산 데이터 병렬 구현을 지정할 수 있습니다.

분산 전략은 `--tensor-model-parallel-size` 및 `--pipeline-model-parallel-size` 인자를 사용하여 구성됩니다.
[Megatron-LM 설명서](https://github.com/NVIDIA/Megatron-LM#distributed-pretraining)에서 분산 전략들에 대해 자세히 알아보십시오.

우리는 (배포 전략이 적용되지 않은) 1개의 GPU에서만 GPT 사전 훈련을 실행할 스크립트  [pretrain_gpt_1GPU.sh](/dli/code/pretrain_gpt_1GPU.sh) 를 준비했습니다.

이 스크립트는 연산 리소스가 이미 할당되었다고 가정합니다. 따라서 실행을 위해 먼저 대화형 세션에서 워커 노드에 연결하여 필요한 GPU를 할당해야 합니다.

## 2.2.1 GPT 사전 훈련 스크립트 확인

리소스를 할당하고 실행하기 전에 스크립트를 살펴보겠습니다. 

모델 아키텍처 및 트레이닝 인자에 주목하십시오. 

In [3]:
# Have a look at the Megaton-LM GPT pretraining execution on 1 GPU script
! cat /dli/code/pretrain_gpt_1GPU.sh

#!/bin/bash

# Distributed training args
NNODES=1
GPUS_PER_NODE=1
TP_SIZE=1
PP_SIZE=1

# Distributed training 
MICRO_BATCH_SIZE=2
GLOBAL_BATCH_SIZE=2

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
NAME="log_1GPU"

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
     

## 2.2.2 GPT 사전 훈련 스크립트 실행

이제 대화형 세션에서 pretrain_gpt_1GPU.sh 스크립트를 실행해 보겠습니다. 실행을 위해서는 다음 3 단계를 따르십시오.
1. 터미널 세션을 시작합니다.
2. srun -N 1 --pty /bin/bash`를 실행하여 대화형 세션을 실행합니다.
3.`bash ./code/pretrain_gpt_1GPU.sh`를 실행하여 1개의 GPU에서 Megatron GPT-3 사전훈련을 실행합니다.


<img src="images/interactive_launch0.png" width="1050"/>

다음 셀을 실행하여 터미널 세션을 여는 링크와 대화형 세션을 실행하는 지침을 생성합니다. 그런 다음 1개의 GPU에서 GPT 사전 훈련 작업을 제출합니다.

In [4]:
%%html

<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 3: Run the megatron gpt3 pretraining on 1 GPU: <font color="green">bash ./code/pretrain_gpt_1GPU.sh</font>
</pre>


GPU 1개에 대한 GPT 사전 훈련이 실행되는 동안, 다음 셀을 실행하여 SLURM 대기열을 확인할 수 있습니다:

In [5]:
# Check the SLURM queue
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 7  slurmpar     bash    admin  R      17:50      1 slurmnode1


`nvidia-smi` 명령을 사용하여 GPU를 확인할 수도 있습니다. 아래 그림과 같이 GPU 0만 활용되고 있습니다. Megatron-LM을 처음 실행할 때는 코드를 컴파일하는 데 약 6분이 소요됩니다. 그 때까지는 GPU 활동을 볼 수 없습니다.

<img src="images/1N_1gpu_utilization.png" width="650"/>

In [6]:
# Check GPU utilization on the master node after Megatron-LM is compiled
! sleep 6m
! nvidia-smi

^C
Fri Mar 28 02:03:25 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   33C    P0    36W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A

## 2.2.3 Megatron-LM 실행 로그 이해

pretrain_gpt_1GPU.sh 스크립트에 지정된 대로 Megatron-LM이 실행되는 월드 크기는 다음과 같아야 합니다.

```
using world size: 1, data-parallel-size: 1, tensor-model-parallel size: 1, pipeline-model-parallel size: 1 
```

![title](images/interactive_launch1.png)

GPT 사전 훈련의 성능을 이해하기 위해 실행 중에 생성된 [로그](./megatron/logs/log_1GPU.txt)를 확인할 수 있습니다.

In [7]:
# Check the Megatron GPT3 pretraining logs.
! grep iteration /dli/megatron/logs/log_1GPU.txt

 iteration       10/     100 | consumed samples:           20 | elapsed time per iteration (ms): 448.0 | learning rate: 6.000E-05 | global batch size:     2 | lm loss: 1.052843E+01 | loss scale: 1.0 | grad norm: 3.027 | number of skipped iterations:   0 | number of nan iterations:   0 |
[Rank 0] (after 10 iterations) memory (MB) | allocated: 1907.2294921875 | max allocated: 10582.0205078125 | reserved: 10922.0 | max reserved: 10922.0
 iteration       20/     100 | consumed samples:           40 | elapsed time per iteration (ms): 273.6 | learning rate: 5.997E-05 | global batch size:     2 | lm loss: 1.004032E+01 | loss scale: 1.0 | grad norm: 3.089 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       30/     100 | consumed samples:           60 | elapsed time per iteration (ms): 273.8 | learning rate: 5.990E-05 | global batch size:     2 | lm loss: 9.632593E+00 | loss scale: 1.0 | grad norm: 2.586 | number of skipped iterations:   0 | number of nan iter

추출에서 출력은 다음과 유사해야 합니다.

```
 iteration      100/     100 | consumed samples:          200 | elapsed time per iteration (ms): 271.6 | learning rate: 5.822E-05 | global batch size:     2 | lm loss: 7.587920E+00 | loss scale: 1.0 | grad norm: 1.468 | number of skipped iterations:   0 | number of nan iterations:   0 |
```   

이번 예제에서는 (글로벌 배치 크기) 2개의 샘플을 처리하는 데 271.6ms의 학습 속도가 소요되었습니다.

좋습니다. 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보하고 나머지 대화형 세션을 취소해 보겠습니다.

In [8]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/*  

----

# 2.3 Megatron-LM GPT 사전 훈련의 멀티 GPU 훈련 실행

이제 대화형 세션에서 사용할 수 있는 2개의 GPU를 활용하여 동일한 이전 훈련 작업을 실행해 보겠습니다. 

`torch.distributed.launch`를 사용하여 2개의 GPU에서 작업을 시작하면 노드당 프로세스 수를 `--nproc_per_node 2`로 설정해야 합니다. 

우리가 실험할 첫 번째 분산 전략은 데이터 병렬 분산 전략으로, 여러 리소스가 사용 가능할 때 기본적으로 Megatron-LM 으로 실행됩니다.
             
단일 GPU에서 이전에 실행된 경우, GPU에 의해 처리된 배치 크기는 2(`--micro-batch-size`에 의해 설정됨)였으며, 이는 글로벌 배치 크기(`--global-batch-size`)에도 해당합니다. 


## 2.3.1 연습: 2개의 GPU에서 Megatron-LM GPT 사전 훈련 실행

다음 셀의 "FIXME"를 수정하여 데이터 병렬 분산을 사용하여 2개의 GPU에서 새로운 Megatron-LM GPT 사전 훈련 실행을 구성해 보겠습니다. 

2개의 GPU를 사용하려면 GPU당 마이크로 배치 크기를 2로 유지하고 글로벌 배치 크기를 4로 2배 늘리면 됩니다. 막히면 언제든지 [솔루션](solutions/ex2.3.ipynb)을 확인하십시오.

각 실행마다 로그 파일 이름(이번 예제에서는 *log_2GPU.txt*)을 변경하겠습니다.

In [11]:
%%writefile /dli/code/pretrain_gpt_2GPU.sh

#!/bin/bash

# Distributed training args
NNODES=1
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1

# Distributed training 
MICRO_BATCH_SIZE=2
GLOBAL_BATCH_SIZE=4

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
NAME="log_2GPU"        

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
            "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
            $EXIT_OPTS \
            "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            "
export LAUNCHER="python -u -m torch.distributed.launch \
            --nproc_per_node $GPUS_PER_NODE \
            --nnodes $NNODES \
            --master_addr $MASTER_ADDR \
            --master_port $MASTER_PORT \
            "

export CMD=" \
            /dli/megatron/Megatron-LM/pretrain_gpt.py \
            --tensor-model-parallel-size $TP_SIZE \
            --pipeline-model-parallel-size $PP_SIZE \
            $GPT_ARGS \
            $OUTPUT_ARGS \
            --save $CHECKPOINT_PATH \
            --data-path $DATA_PATH \
            --data-impl mmap \
            --split 949,50,1 \
            --distributed-backend nccl \
            "

bash -c '$LAUNCHER  $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2GPU.sh


이제 대화형 세션에서 이 스크립트를 실행합니다. 실행을 위해서는 다음 3단계를 따르십시오.
1. 터미널 세션을 시작합니다.
2. `srun -N 1 --pty /bin/bash`를 실행하여 대화형 세션을 실행합니다.
3.`bash ./code/pretrain_gpt_2GPU.sh`를 실행하여 1개의 GPU에서 megatron gpt3 사전 훈련을 실행합니다.

다음 셀을 실행하여 터미널 세션을 여는 링크와 대화형 세션을 실행하는 지침을 가져옵니다. 이후에 2개의 GPU에서 사전 훈련 작업을 제출합니다.

In [12]:
%%html

<pre>
   Step 1: Open a terminal session by following the <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 3: Run the megatron gpt3 pretraining on 1 GPU: <font color="green">bash ./code/pretrain_gpt_2GPU.sh</font>
</pre>

노드 1개에서 GPU 2개에 대한 GPT 사전 훈련이 실행되는 동안, 우리는 SLURM 대기열을 확인할 수 있습니다.

In [13]:
# Check the SLURM queue
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 7  slurmpar     bash    admin  R      26:06      1 slurmnode1
                 8  slurmpar     bash    admin  R       1:52      1 slurmnode1


`nvidia-smi` 명령을 사용하여 GPU 활용 현황을 확인할 수도 있습니다. 아래 그림과 같이 GPU 0과 1이 활용되는 것을 볼 수 있습니다.

<img src="images/1N_2gpus_utilization.png" width="650"/>

In [14]:
# Check GPU utilization on the master node
!nvidia-smi

Fri Mar 28 02:11:14 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   42C    P0   272W / 300W |  12112MiB / 16160MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

## 2.3.2 멀티 GPU Megatron-LM 실행 로그 이해하기

실행 로그를 살펴보겠습니다.

<img src="images/interactive_launch2.png" width="900"/>

Megatron-LM이 실행할 월드 크기는 다음과 같습니다. 

```
world size: 2, data-parallel-size: 2, tensor-model-parallel size: 1, pipeline-model-parallel size: 1
```

2개의 GPU를 사용할 수 있기 때문에 기본적으로 실행되는 분산 전략은 데이터 병렬 전략입니다. 즉, 모델은 두 GPU에 복사되어 서로 다른 데이터 배치를 처리합니다. 

2개의 GPU에 대한 GPT 사전 훈련의 성능을 이해하기 위해, 우리는 실행 중에 생성된 [로그 파일](/dli/megatron/logs/log_2GPU.txt) 을 확인할 수 있습니다.

In [15]:
! grep iteration /dli/megatron/logs/log_2GPU.txt

[Rank 0] (after 10 iterations) memory (MB) | allocated: 1907.2294921875 | max allocated: 10582.0205078125 | reserved: 10922.0 | max reserved: 10922.0
 iteration       10/     100 | consumed samples:           40 | elapsed time per iteration (ms): 820.3 | learning rate: 6.000E-05 | global batch size:     4 | lm loss: 1.050107E+01 | loss scale: 1.0 | grad norm: 6.424 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       20/     100 | consumed samples:           80 | elapsed time per iteration (ms): 464.1 | learning rate: 5.997E-05 | global batch size:     4 | lm loss: 9.977966E+00 | loss scale: 1.0 | grad norm: 2.653 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       30/     100 | consumed samples:          120 | elapsed time per iteration (ms): 470.5 | learning rate: 5.990E-05 | global batch size:     4 | lm loss: 9.543288E+00 | loss scale: 1.0 | grad norm: 2.567 | number of skipped iterations:   0 | number of nan iter

추출 로그에서 2개의 GPU를 사용하는 동안 1개의 GPU와 비교하여 훈련 성능을 확인합니다.

` iteration      100/     100 | consumed samples:          400 | elapsed time per iteration (ms): 363.6 | learning rate: 5.822E-05 | global batch size:     4 | lm loss: 7.500983E+00 | loss scale: 1.0 | grad norm: 1.360 | number of skipped iterations:   0 | number of nan iterations:   0 |`
 
 
소모된 샘플의 수와 해당 훈련 시간에 유의하십시오. 또한 이는 멀티 GPU 시스템에서 바람직한 특성인 거의 선형적인 증가를 보이고 있습니다.

성능에 대해 강사님과 토론해 봅시다 여기서 가장 큰 변화는 같은 시간 동안 처리된 샘플의 수가 더 많다는 것입니다. 따라서 모델이 더 풍부한 데이터 표현을 학습하여 트레이닝을 가속화하는 데 도움이 됩니다.

좋습니다. 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보하고 나머지 대화형 세션을 취소하겠습니다.

In [16]:
# Clean the checkpoints
! rm -rf /dli/megatron/checkpoints/*  

## 2.3.3 모델 배포 시 고려 사항 

텐서 또는 파이프라인 병렬 모드에서 이전 멀티-GPU 스크립트를 실행하려면`--tensor-model-parallel-size` 또는`--pipeline-model-parallel-size` 인자를 사용하여 분포를 구성할 수 있습니다. 

GPU 수에 해당하는 Megatron-LM 훈련의 월드 크기는 그대로 유지되며 데이터-병렬, 텐서-모델-병렬 및 파이프라인-모델-병렬은 구성에 따라 조정해야 합니다. 

```
world size: 2, data-parallel-size: 1, tensor-model-parallel size: 2, pipeline-model-parallel size: 1
or
world size: 2, data-parallel-size: 1, tensor-model-parallel size: 1, pipeline-model-parallel size: 2

```
월드 크기는 데이터 병렬 크기, 텐서 모델 병렬 및 파이프라인 모델 병렬의 곱입니다.

---
<h2 style="color:green;">축하합니다!</h2>

GPU 클러스터에서 GPT-3를 사전 훈련해보았습니다! 훌륭합니다.<br>

다음으로 넘어가기 전에 SLURM 대기열에서 실행 중이거나 대기 중인 작업이 없는지 확인해야 합니다. 
다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다.

In [17]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 7  slurmpar     bash    admin  R      27:17      1 slurmnode1
                 8  slurmpar     bash    admin  R       3:03      1 slurmnode1


여전히 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하여 `scancel`  명령을 사용하여 모든 사용자의 작업을 취소합니다.

In [18]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


다음으로 멀티 노드 분산 구성에 대한 GPT 언어 모델 훈련을 실행할 것입니다. [03_GPT_LM_pretrainings_multinodes.ipynb](03_GPT_LM_pretrainings_multinodes.ipynb)로 이동합니다.